In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.5 MB/s eta 0:00:00


In [ ]:
!pip install pyspark py4j

In [ ]:
import csv
from faker import Faker
import random

fake = Faker()

num_records = 100000

http_methods = ['GET', 'POST', 'PUT', 'DELETE']
response_codes = [200, 301, 404, 500]

file_path = "web_server_logs.csv"

with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ip', 'timestamp', 'method', 'url', 'response_code', 'response_size'])

    for _ in range(num_records):
        ip = fake.ipv4()
        timestamp = fake.date_time_this_year().isoformat()
        method = random.choice(http_methods)
        url = fake.uri_path()
        response_code = random.choice(response_codes)
        response_size = random.randint(100, 10000)

        writer.writerow([ip, timestamp, method, url, response_code, response_size])

print(f"Сгенерировано {num_records} записей и сохранено в {file_path}")

Сгенерировано 100000 записей и сохранено в web_server_logs.csv


In [ ]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import col, to_date, mean, month, year, when

# Создание SparkSession

spark = SparkSession.builder.appName("IpAnalysis").getOrCreate()

# Чтение данных

web_server_df = spark.read.csv("web_server_logs.csv", header=True, inferSchema=True)

In [ ]:
web_server_df.show(20)

+---------------+--------------------+------+--------------------+-------------+-------------+
|             ip|           timestamp|method|                 url|response_code|response_size|
+---------------+--------------------+------+--------------------+-------------+-------------+
|  11.185.21.101|2025-03-26 14:00:...|   GET|    posts/wp-content|          500|         1574|
| 138.235.45.152|2025-05-25 19:58:...|DELETE|      app/categories|          301|         9196|
|   44.2.143.194|2025-08-03 21:33:...|DELETE|       list/category|          500|         8377|
|112.105.205.192|2025-02-03 16:47:...|   GET|           blog/main|          404|         7445|
|167.197.204.253|2025-04-08 23:53:...|  POST|   wp-content/search|          200|         3261|
|  87.235.105.19|2025-08-02 17:56:...|   PUT|      blog/main/tags|          301|         4350|
|  89.186.176.11|2025-03-02 08:53:...|  POST|    category/explore|          301|         6818|
|  211.72.252.42|2025-04-28 22:32:...|  POST|wp-co

In [ ]:
# Регистрация DataFrame
web_server_df.createOrReplaceTempView("logs")

# Задание:
# Сгруппируйте данные по IP и посчитайте количество запросов для каждого IP, выводим 10 самых активных IP
z_df = spark.sql("""
SELECT ip,(COUNT(*)) AS request_count
FROM logs
GROUP BY ip
ORDER BY request_count DESC
""")

print('Top 10 active IP addresses:')
# Показ результатов
z_df.show(10)

Top 10 active IP addresses:
+---------------+-------------+
|             ip|request_count|
+---------------+-------------+
|158.109.166.156|            2|
|    12.81.3.193|            1|
|172.156.127.165|            1|
|   33.38.202.82|            1|
| 121.109.40.230|            1|
| 74.189.115.159|            1|
|  218.62.156.51|            1|
|  73.67.162.196|            1|
|  51.245.82.101|            1|
|  149.92.35.243|            1|
+---------------+-------------+
only showing top 10 rows



In [ ]:
# Задание:
# Сгруппируйте данные по HTTP-методу и посчитайте количество запросов для каждого метода.
http_df = spark.sql("""
SELECT method,(COUNT(*)) AS method_count
FROM logs
GROUP BY method
ORDER BY method_count DESC
""")

print('Request count by HTTP method:')
# Показ результатов
http_df.show()


Request count by HTTP method:
+------+------------+
|method|method_count|
+------+------------+
|   PUT|       25301|
|  POST|       25100|
|   GET|       24846|
|DELETE|       24753|
+------+------------+



In [ ]:
# Задание:
#Профильтруйте и посчитайте количество запросов с кодом ответа 404.
req1_df = spark.sql("""
SELECT (COUNT(*)) AS response_count
FROM logs
WHERE response_code = 404
""")

response_count = req1_df.first()['response_count']

print('Number of 404 response code:',response_count)


Number of 404 response code: 25073


In [27]:
# Задание:
#Сгруппируйте данные по дате и просуммируйте размер ответов, сортируйте по дате.

data_df = spark.sql("""
SELECT DATE(timestamp) AS date,(SUM(response_size)) AS total_response_size
FROM logs
GROUP BY DATE(timestamp)
ORDER BY date ASC
""")

print('Total response size by day:')
# Показ результатов
data_df.show()


Total response size by day:
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2025-01-01|            2393342|
|2025-01-02|            2256660|
|2025-01-03|            2243011|
|2025-01-04|            2445623|
|2025-01-05|            2244198|
|2025-01-06|            2143094|
|2025-01-07|            2267350|
|2025-01-08|            2415279|
|2025-01-09|            2116010|
|2025-01-10|            2279789|
|2025-01-11|            2321200|
|2025-01-12|            2531599|
|2025-01-13|            2579026|
|2025-01-14|            2303737|
|2025-01-15|            2132195|
|2025-01-16|            2198705|
|2025-01-17|            2210819|
|2025-01-18|            2182767|
|2025-01-19|            2208834|
|2025-01-20|            2069093|
+----------+-------------------+
only showing top 20 rows

